# Experiment using model 1

Description


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
from everywhere.datasets.load_data import get_users_data, get_events_data, get_train_data, get_user_friends_data, get_event_attendees_data

Numpy version: 1.24.2
Pandas version: 1.5.3
Everywhere version: 0.0.0


In [ ]:
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Everywhere version: {everywhere.__version__}")

## Model theoretical explanation

This model is based in ...


In [3]:
# Parameters
batch_size = 32
n_layers = 5
#epochs = 10
EPOCHS = 10

## Data prep


In [4]:
data_path = "/Users/diegosanmartin/Desktop/Diego/Dsanmart/everywhere-reco/examples/local_data/"
"""df_events = pd.read_csv(data_path + "events.csv")
df_users = pd.read_csv(data_path + "users.csv")
df_user_friends = pd.read_csv(data_path + "user_friends.csv")
df_event_attendees = pd.read_csv(data_path + "event_attendees.csv")
df_train = pd.read_csv(data_path + "train.csv")"""

'df_events = pd.read_csv(data_path + "events.csv")\ndf_users = pd.read_csv(data_path + "users.csv")\ndf_user_friends = pd.read_csv(data_path + "user_friends.csv")\ndf_event_attendees = pd.read_csv(data_path + "event_attendees.csv")\ndf_train = pd.read_csv(data_path + "train.csv")'

In [7]:
from dotenv import load_dotenv
load_dotenv()
get_train_data(data_path + "train.csv")

Local dataset loaded :)


,user,event,invited,timestamp,interested,not_interested
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0
...,...,...,...,...,...,...
15393,4293103086,2750873665,0,2012-12-08 03:59:43.169000+00:00,0,0
15394,4293103086,4084655790,0,2012-12-08 03:59:43.169000+00:00,0,0
15395,4293103086,598708806,0,2012-12-08 03:59:43.169000+00:00,0,0
15396,4293103086,604179853,0,2012-12-08 03:59:43.169000+00:00,0,0


In [8]:
df_events.head()

,event_id,user_id,start_time,city,state,zip,country,lat,lng,c_1,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,684921758,3647864012,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,1,0,0,0,0,0,0,0,9
1,244999119,3476440521,2012-11-03T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,7
2,3928440935,517514445,2012-11-05T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,12
3,2582345152,781585781,2012-10-30T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,8
4,1051165850,1016098580,2012-09-27T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,9


In [13]:
print(df_event_attendees.shape)
df_event_attendees.head()

(24144, 5)


,event,yes,maybe,invited,no
0,1159822043,1975964455 252302513 4226086795 3805886383 142...,2733420590 517546982 1350834692 532087573 5831...,1723091036 3795873583 4109144917 3560622906 31...,3575574655 1077296663
1,686467261,2394228942 2686116898 1056558062 3792942231 41...,1498184352 645689144 3770076778 331335845 4239...,1788073374 733302094 1830571649 676508092 7081...,NaN
2,1186208412,NaN,3320380166 3810793697,1379121209 440668682,1728988561 2950720854
3,2621578336,NaN,NaN,NaN,NaN
4,855842686,2406118796 3550897984 294255260 1125817077 109...,2671721559 1761448345 2356975806 2666669465 10...,1518670705 880919237 2326414227 2673818347 332...,3500235232


In [28]:
df_users.head()

,user_id,locale,birthyear,gender,joinedAt,location,timezone
0,3197468391,id_ID,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia,480.0
1,3537982273,id_ID,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia,420.0
2,823183725,en_US,1975,male,2012-10-06T03:14:07.149Z,Stratford Ontario,-240.0
3,1872223848,en_US,1991,female,2012-11-04T08:59:43.783Z,Tehran Iran,210.0
4,3429017717,id_ID,1995,female,2012-09-10T16:06:53.132Z,NaN,420.0


In [11]:
df_user_friends.head()

,user,friends
0,3197468391,1346449342 3873244116 4226080662 1222907620 54...
1,3537982273,1491560444 395798035 2036380346 899375619 3534...
2,823183725,1484954627 1950387873 1652977611 4185960823 42...
3,1872223848,83361640 723814682 557944478 1724049724 253059...
4,3429017717,4253303705 2130310957 1838389374 3928735761 71...


In [29]:
df_train = get_friends_attendee_nums(df_train, df_user_friends, df_event_attendees)
df_train

,user,event,invited,timestamp,interested,not_interested,friends,friends_attending,friends_not_attending,friends_maybe_attending,friends_invited
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,3,0,3
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,0,0,1
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0,862,0,0,0,2
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0,862,1,0,0,0
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
15393,4293103086,2750873665,0,2012-12-08 03:59:43.169000+00:00,0,0,480,0,0,1,0
15394,4293103086,4084655790,0,2012-12-08 03:59:43.169000+00:00,0,0,480,1,0,0,1
15395,4293103086,598708806,0,2012-12-08 03:59:43.169000+00:00,0,0,480,1,1,1,3
15396,4293103086,604179853,0,2012-12-08 03:59:43.169000+00:00,0,0,480,0,1,1,0


In [30]:
df_train = get_event_attendee_nums(df_train, df_event_attendees)
df_train

,user,event,invited_x,timestamp,interested,not_interested,friends,friends_attending,friends_not_attending,friends_maybe_attending,friends_invited,yes,maybe,invited_y,no,users_yes,users_no,users_maybe,users_invited_count
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,3,0,3,671846165 3400950561 3622121962 2914089126 427...,3777634824 707808605,243449741 2705495661 2053207729 2228554219 229...,3648238803 3898796217 2892558795 2522528816 65...,8,25,2,23
1,1906220044,1918771225,0,2012-10-01 05:08:48.533000+00:00,0,0,930,1,0,0,0,671846165 3400950561 3622121962 2914089126 427...,3777634824 707808605,243449741 2705495661 2053207729 2228554219 229...,3648238803 3898796217 2892558795 2522528816 65...,8,25,2,23
2,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,0,0,1,2897626992 3265030005 593212332 1566080870 171...,4221735584 821549713 1573381076 939778463 4110...,2823348035 1461073321 1603391812 1511769431 35...,3928943237 1419223904,10,2,6,122
3,1302145719,1502284248,0,2012-09-30 13:31:33.569000+00:00,0,0,2265,0,0,0,6,2897626992 3265030005 593212332 1566080870 171...,4221735584 821549713 1573381076 939778463 4110...,2823348035 1461073321 1603391812 1511769431 35...,3928943237 1419223904,10,2,6,122
4,3194014105,1502284248,0,2012-10-01 17:20:14.536000+00:00,0,0,619,0,0,0,0,2897626992 3265030005 593212332 1566080870 171...,4221735584 821549713 1573381076 939778463 4110...,2823348035 1461073321 1603391812 1511769431 35...,3928943237 1419223904,10,2,6,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15393,4293103086,2750873665,0,2012-12-08 03:59:43.169000+00:00,0,0,480,0,0,1,0,2485745356 3459443363 1980282349 910372895 349...,3485389656 77305317 1772115747 3398850624 1976...,4089136524 3084736394 4069540035 3458310857 31...,2918390324 2079957246 3050124649 123025466 402...,111,26,81,477
15394,4293103086,4084655790,0,2012-12-08 03:59:43.169000+00:00,0,0,480,1,0,0,1,3018307805 1273322553 1201783345 3145453106 21...,3539902602 299607830 1632488566 3226370688 222...,3641702874 13080990 4118624064 1764266106 1354...,1432663397 1266968360,37,2,37,313
15395,4293103086,598708806,0,2012-12-08 03:59:43.169000+00:00,0,0,480,1,1,1,3,2613828807 2839000364 3251647249 3706041447 92...,1037388933 3721658035 1293968571 4280152220 18...,685339129 3485457810 1790249421 476313752 4087...,3819183717 3058043461 2274217134 1790896391 84...,181,14,66,696
15396,4293103086,604179853,0,2012-12-08 03:59:43.169000+00:00,0,0,480,0,1,1,0,2667969956 773828719 610302199 3016949943 2169...,1772475575 3268437113 98881171 2036663201 1443...,4175140272 1803249223 2535143415 1981386687 67...,1639911140 3655515536 3581056540 3432488575 37...,436,1340,199,842


In [10]:
df_events.columns

Index(['event_id', 'user_id', 'start_time', 'city', 'state', 'zip', 'country',
       'lat', 'lng', 'c_1',
       ...
       'c_92', 'c_93', 'c_94', 'c_95', 'c_96', 'c_97', 'c_98', 'c_99', 'c_100',
       'c_other'],
      dtype='object', length=110)

In [31]:
df_events_clean = df_events.drop(columns=['user_id', 'start_time', 'city', 'state', 'zip', 'country', 'lat', 'lng'])
# Merge with events
df_train = pd.merge(df_train, df_events_clean, how='inner', left_on='event', right_on='event_id')
df_train = df_train.drop(columns=['invited_y', 'timestamp', 'yes', 'maybe', 'no'])
df_train

,user,event,invited_x,interested,not_interested,friends,friends_attending,friends_not_attending,friends_maybe_attending,friends_invited,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,3044012,1918771225,0,0,0,862,0,3,0,3,...,0,0,0,0,0,0,0,0,0,2
1,1906220044,1918771225,0,0,0,930,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,3044012,1502284248,0,0,0,862,0,0,0,1,...,0,0,0,0,0,0,0,0,0,24
3,1302145719,1502284248,0,0,0,2265,0,0,0,6,...,0,0,0,0,0,0,0,0,0,24
4,3194014105,1502284248,0,0,0,619,0,0,0,0,...,0,0,0,0,0,0,0,0,0,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15393,4293103086,2750873665,0,0,0,480,0,0,1,0,...,0,0,0,0,0,0,0,0,0,80
15394,4293103086,4084655790,0,0,0,480,1,0,0,1,...,0,0,0,0,0,0,0,0,0,80
15395,4293103086,598708806,0,0,0,480,1,1,1,3,...,0,0,0,0,0,0,0,0,0,60
15396,4293103086,604179853,0,0,0,480,0,1,1,0,...,0,0,0,0,0,0,0,0,0,24


In [32]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15398 entries, 0 to 15397
Columns: 116 entries, user to c_other
dtypes: int64(116)
memory usage: 13.7 MB


In [20]:
df_train.columns[:20]

Index(['user', 'event', 'invited_x', 'interested', 'not_interested', 'friends',
       'friends_attending', 'friends_not_attending', 'friends_maybe_attending',
       'friends_invited', 'yes', 'maybe', 'no', 'users_yes', 'users_no',
       'users_maybe', 'users_invited_count', 'event_id', 'c_1', 'c_2'],
      dtype='object')

In [25]:
df_train.no

0        3648238803 3898796217 2892558795 2522528816 65...
1        3648238803 3898796217 2892558795 2522528816 65...
2                                    3928943237 1419223904
3                                    3928943237 1419223904
4                                    3928943237 1419223904
                               ...                        
15393    2918390324 2079957246 3050124649 123025466 402...
15394                                1432663397 1266968360
15395    3819183717 3058043461 2274217134 1790896391 84...
15396    1639911140 3655515536 3581056540 3432488575 37...
15397                                                  NaN
Name: no, Length: 15398, dtype: object

## Balance dataset

In [15]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler
from collections import Counter

In [33]:
data=df_train.copy()
X = data.drop('interested', axis = 1)
y = data['interested']
print('Original dataset shape {}'.format(Counter(y)))
cc = ClusterCentroids(random_state=42)
X_res, y_res = cc.fit_resample(X, y)
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({0: 11267, 1: 4131})


c:\Users\34629\OneDrive - IE Students\BCSAI\3rd year\2nd cuatri\recommenders\everywhere-reco\venv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Resampled dataset shape Counter({0: 4131, 1: 4131})


In [34]:
#shuffle the data
final = pd.merge(X_res, y_res, left_index=True, right_index=True)
final = final.iloc[np.random.permutation(final.index)].reset_index(drop=True)
final

,user,event,invited_x,not_interested,friends,friends_attending,friends_not_attending,friends_maybe_attending,friends_invited,users_yes,...,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other,interested
0,1696361679,496131340,0,0,1003,0,0,0,5,19,...,0,0,0,0,0,0,0,0,11,1
1,3597496510,1645469702,0,0,706,0,0,0,1,12,...,0,0,0,0,0,0,0,0,36,1
2,2024676800,1540847076,0,0,2434,0,0,0,7,60,...,0,0,0,0,0,0,0,0,30,0
3,3105505306,2768238024,0,0,326,0,0,0,10,37,...,0,0,0,0,0,0,0,0,70,0
4,615123889,850734249,0,0,1333,0,0,0,3,133,...,0,0,0,0,0,0,0,0,79,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8257,677234912,991427858,0,0,597,0,0,0,0,17,...,0,0,0,0,0,0,0,0,9,1
8258,2633494537,3363493036,0,0,2824,0,0,0,5,4,...,0,0,0,0,0,0,0,0,25,0
8259,2328965162,673098017,0,0,2680,0,1,0,40,20,...,0,0,0,0,0,0,0,0,30,0
8260,4059273005,2874223951,0,0,1447,0,0,0,11,13,...,0,0,0,0,0,0,0,0,55,0


# Model

In [43]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier



In [37]:
#split train and test
X = final.drop('interested', axis = 1)
y = final['interested']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)





In [40]:
#random forest
rf = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
#with probabilities
y_pred_proba = rf.predict_proba(X_test)[:,1]

In [41]:
#accuracy
print("Accuracy: %.2f%%" % (accuracy_score(y_test, y_pred) * 100.0))
#confusion matrix
print(confusion_matrix(y_test, y_pred))
#roc_auc_score
print("ROC AUC score: %.2f%%" % (roc_auc_score(y_test, y_pred_proba) * 100.0))

Accuracy: 74.33%
[[1097  263]
 [ 437  930]]
ROC AUC score: 83.64%


In [55]:
#lightgbm
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
#with probabilities
y_pred_proba = clf.predict_proba(X_test)[:,1]


In [57]:
#accuracy
print("Accuracy: %.2f%%" % (accuracy_score(y_test, y_pred) * 100.0))
#confusion matrix
print(confusion_matrix(y_test, y_pred))
#roc_auc_score
print("ROC AUC score: %.2f%%" % (roc_auc_score(y_test, y_pred_proba) * 100.0))

Accuracy: 81.66%
[[1080  280]
 [ 220 1147]]
ROC AUC score: 89.03%


## Evaluation


## Summary
